PyGameのインストール

sudo apt-get install libsdl-dev libsdl-image1.2-dev libsdl-mixer1.2-dev libsdl-ttf2.0-dev

sudo apt-get install libportmidi-dev

pip3 install pygame

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import os
import subprocess
from jetbot import Robot
import time
import threading
import sys
from tkinter import font
from torch2trt import torch2trt
os.environ['DISPLAY'] = ":0.0"
print(os.environ['DISPLAY'])
from tkinter import *
import pygame
# TensorRT対応の改造
from torch2trt import TRTModule 


:0.0
pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Joystick:
    def __init__(self, root):
        self.root = root
        pygame.init()
        if(pygame.joystick.get_count() < 1):
            print("Please connect a joystick.\n")
            #self.quit()
        else:
            print("Find joysrick!\n")
            joystick = pygame.joystick.Joystick(0)
            joystick.init()
        self.root.after(0, self.find_events)

    def find_events(self):
        events = pygame.event.get()
        joystick = pygame.joystick.Joystick(0)
        lAxisX = joystick.get_axis(0)
        lAxisY = joystick.get_axis(1)
        rAxisX = joystick.get_axis(3)
        rAxisY = joystick.get_axis(4)
        RB = joystick.get_button(5)  
        LB = joystick.get_button(4)
        BACK = joystick.get_button(6)
        START = joystick.get_button(7)
        Y = joystick.get_button(3)
        X = joystick.get_button(2)
        B = joystick.get_button(1)
        A = joystick.get_button(0)
        for event in events:
            # event type for pressing any of the joystick buttons down
            if event.type == pygame.JOYBUTTONDOWN:
                if RB == 1:
                    self.root.event_generate('<<RB>>')
                if LB == 1:
                    self.root.event_generate('<<LB>>')
                if BACK == 1:
                    self.root.event_generate('<<BACK>>')
                if START == 1:
                    self.root.event_generate('<<START>>')
                if Y == 1:
                    self.root.event_generate('<<Y>>')
                if X == 1:
                    self.root.event_generate('<<X>>')
                if A == 1:
                    self.root.event_generate('<<A>>')
                if B == 1:
                    self.root.event_generate('<<B>>')

            if event.type == pygame.JOYAXISMOTION:
                sys.stdout.write("\rlX=%f,lY=%f,rX=%f,rY=%f," % (lAxisX,lAxisY,rAxisX,rAxisY))
                sys.stdout.flush()
        

        ## return to check for more events in a moment
        self.root.after(20, self.find_events)

 sudo apt-get install python3-pil python3-pil.imagetk

In [ ]:
import tkinter as tk
import cv2
from PIL import Image,ImageTk
import numpy as np
import threading
import datetime

# config
PRE_WIDTH = 720
PRE_HEIGHT = 720
width = 224
height = 224
fps = 10
capture_width = 3280
capture_height = 2464
running = False
root=tk.Tk()
root.title("camera")
root.attributes("-fullscreen", True)
root.resizable(width=False, height=False)
joystick = Joystick(root)
canvas=tk.Canvas(root, width=1280, height=720, bg="white")
canvas.pack()
aiflag = False
robotflag = False
DIR_FREE = "/home/jetbot/Notebooks/STEM/dataset/free/"
DIR_BLOCKED = "/home/jetbot/Notebooks/STEM/dataset/blocked/"
DIR_DATASET = "/home/jetbot/Notebooks/STEM/datasetblocked/"
NUM_EPOCHS = 30
BEST_MODEL_PATH = '/home/jetbot/Notebooks/STEM/best_model.pth'
BEST_MODEL_TRT_PATH = '/home/jetbot/Notebooks/STEM/best_model_trt.pth'
    
def camera():
    print('camera start')
    try:
        global c, w, h, img
        c = cv2.VideoCapture(_gst_str(), cv2.CAP_GSTREAMER)
        w, h= c.get(cv2.CAP_PROP_FRAME_WIDTH), c.get(cv2.CAP_PROP_FRAME_HEIGHT)
        print('open image [width:'+str(w)+'px height:'+str(h)+'px]')
    except:
        import sys
        print("error-----")
        print(sys.exec_info()[0])
        print(sys.exec_info()[1])
        '''終了時の処理はここでは省略します。
        c.release()
        cv2.destroyAllWindows()'''

def _gst_str():
    return 'nvarguscamerasrc ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
        capture_width, capture_height, fps, width, height)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    mean = 255.0 * np.array([0.485, 0.456, 0.406])
    stdev = 255.0 * np.array([0.229, 0.224, 0.225])
    normalize = torchvision.transforms.Normalize(mean, stdev)
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x
    
def recognize(image):
    global model, robot
    #print("recognizez")
    x = preprocess(image)
    y = model(x)

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    text_blocked_level["text"] = "回転の確率: " + "{0:.2f}".format(prob_blocked*100) + "%"
    sys.stdout.write("\rblockded %2f" % (prob_blocked*100))
    sys.stdout.flush()
        
    #print(prob_blocked)

    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
        
    time.sleep(0.001)

def training():
    text_info["text"] = "[1/5] データセットの読み込み"
    dataset = datasets.ImageFolder(
        'dataset',
        transforms.Compose([
            transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    )
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])
        
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=4
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=4
    )
        
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
    device = torch.device('cuda')
    model = model.to(device)
        
    text_info["text"] = "[2/5] 学習の開始"
    
    best_accuracy = 0.0
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(NUM_EPOCHS):

        for images, labels in iter(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()

        test_error_count = 0.0
        for images, labels in iter(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))

        test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
        print('%d: %f' % (epoch, test_accuracy))
        text_info["text"] = "[3/5] 回数: " + str(epoch) + "/29 精度: " + str(test_accuracy)
        if test_accuracy > best_accuracy:
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            best_accuracy = test_accuracy
        
    print("[4/5] TensorRTへの変換")
    text_info["text"] = "[4/5] TensorRTへの変換"
    model.load_state_dict(torch.load(BEST_MODEL_PATH))
    device = torch.device('cuda')
    model = model.to(device)
    model = model.cuda().eval().half()
    data = torch.randn((1, 3, 224, 224)).cuda().half()
    model_trt = torch2trt(model, [data], fp16_mode=True)
    print("[5/5] TensorRTモデルの保存")
    text_info["text"] = "[5/5] TensorRTモデルの保存"
    torch.save(model_trt.state_dict(), BEST_MODEL_TRT_PATH)
    print("学習が終了しました。")
    text_info["text"] = "学習が終了しました。"
    
def checkDataset():
    try:
        os.makedirs(DIR_FREE)
        os.makedirs(DIR_BLOCKED)
        print('Created dataset folder!')
    except FileExistsError:
        print('Directories not created becasue they already exist')
          
def update():
    global running, image, aiflag, canvas
    if aiflag:
        print("running:" + str(running))
    while running:
        ret, image = c.read()
        if aiflag:
            print("ret:" + str(ret))
        if ret:
            img = ImageTk.PhotoImage(Image.fromarray(cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (int(PRE_WIDTH), int(PRE_HEIGHT)))))
            canvas.create_image(PRE_WIDTH/2,PRE_HEIGHT/2,image=img,tag="img")
            if aiflag:
                recognize(image)
        else:
            if aiflag:
                print("break")
            break

def start():
    global thread, running
    print("START")
    running = True
    thread = threading.Thread(target=update)
    thread.start()

def stop():
    global c, thread, running
    print("STOP")
    running = False
    thread.join()

def load():
    global model, device, aiflag, robot, robotflag
    stop()
    print("ai.load")
    text_info["text"] = "ai load"
    model = torchvision.models.alexnet(pretrained=False)
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
    print("call robot")
    text_info["text"] = "call robot"
    robot = Robot()
    robotflag = True
    print("load model")
    text_info["text"] = "load model"
    model.load_state_dict(torch.load(BEST_MODEL_PATH))
    #model_trt = TRTModule()
    #model_trt.load_state_dict(torch.load(BEST_MODEL_PATH))
    text_info["text"] = "Finish loading"
    device = torch.device('cuda')
    model = model.to(device)
    #model = model_trt.to(device)
    #model = model_trt.eval()
    text_info["text"] = "Move to cuda"
    aiflag = True
    #cameraStart()
    start()
    
        
def blockFunc(event):
    cv2.imwrite('{}_{}.{}'.format(DIR_BLOCKED, datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'), "jpg"), image)
    blocked_num = getNumFile(DIR_BLOCKED)
    text_blocked["text"] = "障害: " + str(blocked_num)

def freeFunc(event):
    cv2.imwrite('{}_{}.{}'.format(DIR_FREE, datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'), "jpg"), image)
    free_num = getNumFile(DIR_FREE)
    text_free["text"] = "走行: " + str(free_num)
    
def trainFunc(event):
    text_info["text"] = "AIの学習"
    training_thread = threading.Thread(target=training)
    training_thread.start()
    
def runFunc(event):
    global aiflag
    text_info["text"] = "AIの呼び出し"
    print("AI")
    
    load_thread = threading.Thread(target=load)
    load_thread.start()
    
def getNumFile(dir):
    return len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name))])


def backFunc(event):
    global running, robot, robotflag
    print("BACK")
    if robotflag:
        print("ROBOT")
        robot = Robot()
    running = False

    c.release()
    cv2.destroyAllWindows()
    
    root.quit()
    root.destroy()
    pygame.quit()
    
    exit()

# テキスト表示
font1 = font.Font(family='Helvetica', size=25, weight='bold')
text_info = tk.Label(root, text="起動", fg="black", bg="white", font=font1)
text_info.place(x=750, y=5)

font2 = font.Font(family='Helvetica', size=40, weight='bold')
blocked_num = getNumFile(DIR_BLOCKED)
text_blocked = tk.Label(root, text="障害: " + str(blocked_num), fg="red", bg="white", font=font2)
text_blocked.place(x=800, y=200)
free_num = getNumFile(DIR_FREE)
text_free = tk.Label(root, text="走行: " + str(free_num), fg="blue", bg="white", font=font2)
text_free.place(x=800, y=300)


text_blocked_level = tk.Label(root, text="障害の確率: ", fg="blue", bg="white", font=font2)
text_blocked_level.place(x=750, y=500)

checkDataset()
camera()
start()
root.bind('<<BACK>>', backFunc)
root.bind('<<X>>', freeFunc)
root.bind('<<B>>', blockFunc)
root.bind('<<Y>>', trainFunc)
root.bind('<<A>>', runFunc)

root.mainloop()

Find joysrick!

Directories not created becasue they already exist
camera start
open image [width:224.0px height:224.0px]
START
lX=0.000000,lY=0.000000,rX=0.000000,rY=-0.000061,AI
STOP
ai.load
call robot
load model
START


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-594f23913413>", line 176, in update
    print("running:" + running)
TypeError: must be str, not bool



In [ ]:
robot = Robot()

In [ ]:
!free -h